In [ ]:
import tweepy
import csv
from tweepy import OAuthHandler
 
consumer_key = '9fYKitXCiZQYIjxeuZmME8qeE'
consumer_secret = 'Ey8cs2nTAtS9t7pHkY0FXc2ynnR9rZdKB8qosCbtsXAy1vcAzc'
access_token = '478778262-6GodKjQB2FIp9nHHFJ0fSsg0v8YBe6GNJYJZxYvo'
access_secret = 'yCeC1SdoEH540UozGAWFIs7UL0CIpVGwWGTNk7gWytNIg'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth,wait_on_rate_limit = True)
csvFile = open('result.csv', 'a')
count=0

#Use csv writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,
                           q = "sridevi",
                           rpp=10,
                           tweet_mode='extended',
                           lang = "en").items(1000):

    # Write a row to the CSV file. I use encode UTF-8
    if 'retweeted_status' not in dir(tweet):
        csvWriter.writerow([tweet.created_at, tweet.full_text.encode('utf-8')])
        count=count+1;
        print(tweet.full_text)
    
    
csvFile.close()
print(count);